# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [ ]:
!pip install yelpapi
!pip install tqdm

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open('/Users/brend/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [3]:
# You can display the keys, but do NOT display the values!
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

### Define Search Terms and File Paths


In [5]:
# set our API call parameters and filename before the first call
location = 'Atlanta, GA 30033 '
term = 'mexican'

In [6]:
# Programmatically obtain the city name only from the location string to use in our file name
location.split(',')[0]

'Atlanta'

In [7]:
## Specify folder for saving data
FOLDER = 'Data/'
# This will make the Data folder for us if we don't have one
os.makedirs(FOLDER, exist_ok = True)


# Specifying JSON_FILE filename (can include a folder) to include the location and search term
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

#Verify the name of the JSON_FILE
JSON_FILE

'Data/Atlanta-mexican.json'

In [8]:
# We can extract the folder name from the file path 
folder = os.path.dirname(JSON_FILE)
folder

'Data'

### Check if Json File exists and Create it if it doesn't

In [10]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON folder name is not empty: (We have included a folder in our filepath)
    if len(folder)>0:
        # create the folder (In our case, the folder is Data)
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Creating and saving empty list to file.")

    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Atlanta-mexican.json already exists.


### Load JSON FIle and account for previous results

### Make the first API call to explore the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [11]:
# use our yelp_api variable's search_query method to perform our API call
# for help with any function, go inside () and click shift + tab
# we MUST include location (or lat/long) but we can include other arguments as well

results = yelp_api.search_query(location = location, term = term)


In [12]:
# Check data type of the results
type(results)

dict

In [13]:
# Check the length of the results
len(results)

3

In [14]:
# List the keys of the dictionary
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [15]:
# Explore each key
results['businesses']

[{'id': '9HjUwnEL5WIlv7RnnzF7CQ',
  'alias': 'taqueria-el-vecino-decatur',
  'name': 'Taqueria El Vecino',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/_QwrEmkWJba35r4Qt13V7w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/taqueria-el-vecino-decatur?adjust_creative=KJtcufKUS887p24u6rjVIQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KJtcufKUS887p24u6rjVIQ',
  'review_count': 272,
  'categories': [{'alias': 'mexican', 'title': 'Mexican'},
   {'alias': 'bbq', 'title': 'Barbeque'}],
  'rating': 3.8,
  'coordinates': {'latitude': 33.8239171, 'longitude': -84.2939897},
  'transactions': ['delivery', 'pickup'],
  'price': '$$',
  'location': {'address1': '2743 Lavista Rd',
   'address2': '',
   'address3': '',
   'city': 'Decatur',
   'zip_code': '30033',
   'country': 'US',
   'state': 'GA',
   'display_address': ['2743 Lavista Rd', 'Decatur, GA 30033']},
  'phone': '+14049290301',
  'display_phone': '(404) 929-0301',
  'distance': 1361.56

In [16]:
# Explore each key
results['total']

485

In [17]:
# Explore each key
results['region']

{'center': {'longitude': -84.287109375, 'latitude': 33.813088376606714}}

- Where is the actual data we want to save?

In [19]:
# What is the datatype for the businesses?
type(results['businesses'])

list

In [20]:
# Display first result from the list of dictionaries
results['businesses'][0]

{'id': '9HjUwnEL5WIlv7RnnzF7CQ',
 'alias': 'taqueria-el-vecino-decatur',
 'name': 'Taqueria El Vecino',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/_QwrEmkWJba35r4Qt13V7w/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/taqueria-el-vecino-decatur?adjust_creative=KJtcufKUS887p24u6rjVIQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KJtcufKUS887p24u6rjVIQ',
 'review_count': 272,
 'categories': [{'alias': 'mexican', 'title': 'Mexican'},
  {'alias': 'bbq', 'title': 'Barbeque'}],
 'rating': 3.8,
 'coordinates': {'latitude': 33.8239171, 'longitude': -84.2939897},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '2743 Lavista Rd',
  'address2': '',
  'address3': '',
  'city': 'Decatur',
  'zip_code': '30033',
  'country': 'US',
  'state': 'GA',
  'display_address': ['2743 Lavista Rd', 'Decatur, GA 30033']},
 'phone': '+14049290301',
 'display_phone': '(404) 929-0301',
 'distance': 1361.563323329334}

In [21]:
## We can convert the a list of dictionaries  to a datafram
df = pd.DataFrame(results['businesses'])
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,9HjUwnEL5WIlv7RnnzF7CQ,taqueria-el-vecino-decatur,Taqueria El Vecino,https://s3-media3.fl.yelpcdn.com/bphoto/_QwrEm...,False,https://www.yelp.com/biz/taqueria-el-vecino-de...,272,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.8,"{'latitude': 33.8239171, 'longitude': -84.2939...","[delivery, pickup]",$$,"{'address1': '2743 Lavista Rd', 'address2': ''...",+14049290301,(404) 929-0301,1361.563323
1,GfSQJ3Ex7IjomEAav9Mwzw,la-parrilla-mexican-restaurant-atlanta-2,La Parrilla Mexican Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/xDfJMt...,False,https://www.yelp.com/biz/la-parrilla-mexican-r...,437,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.7,"{'latitude': 33.8147766440452, 'longitude': -8...",[delivery],$$,"{'address1': '2945 North Druid Hills Rd', 'add...",+16784033831,(678) 403-3831,2198.475924
2,HKZ0dUt9sUUZVYlzwjX0ig,los-ninos-taqueria-decatur-2,Los Ninos Taqueria,https://s3-media2.fl.yelpcdn.com/bphoto/pphWgJ...,False,https://www.yelp.com/biz/los-ninos-taqueria-de...,75,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 33.7709975, 'longitude': -84.2922...",[],$,"{'address1': '307 E College Ave B', 'address2'...",,,4704.697534
3,V86AJK3xytBv2C0bwi9_Ow,la-pastorcita-atlanta,La Pastorcita,https://s3-media2.fl.yelpcdn.com/bphoto/QwqXsI...,False,https://www.yelp.com/biz/la-pastorcita-atlanta...,505,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.1,"{'latitude': 33.84308, 'longitude': -84.32924}","[delivery, pickup]",$,"{'address1': '3304 Buford Hwy NE', 'address2':...",+16787058162,(678) 705-8162,5117.990718
4,QN4FMM1-J4Me5hu-ro85EA,taqueria-la-carreta-atlanta,Taqueria La Carreta,https://s3-media3.fl.yelpcdn.com/bphoto/8sFvhY...,False,https://www.yelp.com/biz/taqueria-la-carreta-a...,60,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.3,"{'latitude': 33.8424612046155, 'longitude': -8...",[delivery],$$,"{'address1': '3375 Buford Hwy NE', 'address2':...",+16787771447,(678) 777-1447,4914.941348


In [22]:
## How many did we get the details for?
len(df)

20

- Calculate how many pages of results needed to cover the total_results

In [24]:
# How many pages are needed?
n_pages = results['total']/len(df)
n_pages

24.25

In [25]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(n_pages)
n_pages

25

# Create the JSON FILE, give it a programmatic name
# Save our first page of results to the JSON file

In [26]:
## Read in results in progress file and check the length
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

In [27]:
# Preview the contents of the json file (hint, we expect this to be empty at this point)
previous_results

[]

In [28]:
# Determine how many results are present already (hint, we expect this to be 0 at this point
len(previous_results)

0

In [29]:
# Make a loop to extract data from each page and save it to the JSON_FILE

for i in tqdm_notebook(range(n_pages)): 
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
     
       
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results to use as offset for query
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location = location, term = term, offset = n_results)  # Notice we added the offset to our search_query
        

        ## append new results and save to file  (extend is similar to append, but adds a list to a list)
        previous_results.extend(results['businesses'])

        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.

    except Exception as e:
        print(' [!] ERROR', e)

  0%|          | 0/25 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [30]:
# Read the json file in as a df
df = pd.read_json(JSON_FILE)

In [31]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,9HjUwnEL5WIlv7RnnzF7CQ,taqueria-el-vecino-decatur,Taqueria El Vecino,https://s3-media3.fl.yelpcdn.com/bphoto/_QwrEm...,False,https://www.yelp.com/biz/taqueria-el-vecino-de...,272,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.8,"{'latitude': 33.8239171, 'longitude': -84.2939...","[delivery, pickup]",$$,"{'address1': '2743 Lavista Rd', 'address2': ''...",+14049290301,(404) 929-0301,1361.563323
1,GfSQJ3Ex7IjomEAav9Mwzw,la-parrilla-mexican-restaurant-atlanta-2,La Parrilla Mexican Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/xDfJMt...,False,https://www.yelp.com/biz/la-parrilla-mexican-r...,437,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.7,"{'latitude': 33.8147766440452, 'longitude': -8...",[delivery],$$,"{'address1': '2945 North Druid Hills Rd', 'add...",+16784033831,(678) 403-3831,2198.475924
2,HKZ0dUt9sUUZVYlzwjX0ig,los-ninos-taqueria-decatur-2,Los Ninos Taqueria,https://s3-media2.fl.yelpcdn.com/bphoto/pphWgJ...,False,https://www.yelp.com/biz/los-ninos-taqueria-de...,75,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 33.7709975, 'longitude': -84.2922...",[],$,"{'address1': '307 E College Ave B', 'address2'...",,,4704.697534
3,V86AJK3xytBv2C0bwi9_Ow,la-pastorcita-atlanta,La Pastorcita,https://s3-media2.fl.yelpcdn.com/bphoto/QwqXsI...,False,https://www.yelp.com/biz/la-pastorcita-atlanta...,505,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.1,"{'latitude': 33.84308, 'longitude': -84.32924}","[pickup, delivery]",$,"{'address1': '3304 Buford Hwy NE', 'address2':...",+16787058162,(678) 705-8162,5117.990718
4,QN4FMM1-J4Me5hu-ro85EA,taqueria-la-carreta-atlanta,Taqueria La Carreta,https://s3-media3.fl.yelpcdn.com/bphoto/8sFvhY...,False,https://www.yelp.com/biz/taqueria-la-carreta-a...,60,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.3,"{'latitude': 33.8424612046155, 'longitude': -8...",[delivery],$$,"{'address1': '3375 Buford Hwy NE', 'address2':...",+16787771447,(678) 777-1447,4914.941348


In [32]:
# Verify the .info 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             485 non-null    object 
 1   alias          485 non-null    object 
 2   name           485 non-null    object 
 3   image_url      485 non-null    object 
 4   is_closed      485 non-null    bool   
 5   url            485 non-null    object 
 6   review_count   485 non-null    int64  
 7   categories     485 non-null    object 
 8   rating         485 non-null    float64
 9   coordinates    485 non-null    object 
 10  transactions   485 non-null    object 
 11  price          373 non-null    object 
 12  location       485 non-null    object 
 13  phone          485 non-null    object 
 14  display_phone  485 non-null    object 
 15  distance       485 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 57.4+ KB


In [33]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Atlanta-mexican.csv.gz'

In [34]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [35]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 474,685 Bytes
CSV.GZ FILE: 70,455 Bytes
the csv.gz is 6.737421048896459 times smaller!


## Next Class: Processing the Results and Mapping 